In [6]:
import yagmail
import pandas as pd
import os
import glob

In [ ]:
# === НАСТРОЙКИ ===
your_email = "" # ваша почта
app_password = "" # ваш пароль приложения (настраивается через yandex id в разделе безопасность)
folder_with_attachments = "attachments" # название папки с вложениями
cc_list = [""]  # ← адреса в копии

In [ ]:
# Текст письма
body_text = """
Какой-то текст
"""

In [9]:
# Авторизация
yag = yagmail.SMTP(user=your_email, password=app_password, host="smtp.yandex.ru", port=465, smtp_ssl=True)

In [ ]:
# Загрузка данных
df = pd.read_excel("recipients.xlsx")

# === Предварительный сбор адресов и вложений ===
recipients_data = []

for index, row in df.iterrows():
    email = row["Email"]
    identifier = str(row["Идентификатор"]).strip()

    # Поиск всех PDF-файлов в подпапке
    search_path = os.path.join(folder_with_attachments, identifier, "*.pdf")
    attachments = glob.glob(search_path)
    attachments = [f for f in attachments if f.lower().endswith(".pdf")]

    recipients_data.append({
        "email": email,
        "identifier": identifier,
        "attachments": attachments
    })

# === Печать таблицы перед отправкой ===
print("📋 Предпросмотр рассылки и найденных вложений:")
for r in recipients_data:
    print(f"\n📨 {r['email']} (ID: {r['identifier']})")
    if r['attachments']:
        for f in r['attachments']:
            print("  -", os.path.basename(f))
    else:
        print("  ⚠️ Вложений не найдено")

In [ ]:
# Рассылка писем
for index, row in df.iterrows():
    email = row["Email"]
    identifier = row["Идентификатор"]

    # Поиск файлов
    attachments = glob.glob(os.path.join(folder_with_attachments, identifier, "*.pdf"))

    if not attachments:
        print(f"⚠️ Нет вложений для: {email}")
        continue

    # Отправка письма
    try:
        yag.send(
            to=email,
            cc=cc_list,
            subject="", # здесь тема письма
            contents=body_text,
            attachments=attachments
        )
        print(f"✅ Успешно отправлено: {email}")
    except Exception as e:
        print(f"❌ Ошибка при отправке на {email}: {str(e)}")

print("📬 Рассылка завершена.")